In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
#import wandb

#from wandb.integration.keras import WandbMetricsLogger

os.environ["KERAS_BACKEND"] = "torch"

#os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


In [ ]:
import keras
import keras_tuner as kt
keras.utils.set_random_seed(812)

In [ ]:
#wandb.login()

In [ ]:
#run = wandb.init(
#    dir = "../results/syntheticdata/",
#    project = "indexes_subsequence_lstm"
#)

In [ ]:
#create synthetic multivariate time series random uniform integers
n = 1000
k = 3
data = np.random.randint(1, 6, (k, n)) #k timeseries of length n

data_scenario1 = data.copy()

#select random timewindows of length 5 without overlapping
motif_indexes_scenario1 = []
motif_pattern = [1,1,1,1,1]
p = len(motif_pattern)

In [ ]:
max_interval = n//50
print("Max interval between motif", max_interval)
last_index = 0
while True:
    min_interval = p + 10
    index_interval = np.random.randint(min_interval, p + max_interval)
    if last_index + index_interval + p > n:
        break
    last_index = last_index + index_interval
    motif_indexes_scenario1.append(last_index)

print("Planted motifs:", len(motif_indexes_scenario1))
print(motif_indexes_scenario1)
for i in motif_indexes_scenario1:
    for j in range(k):
        data_scenario1[j, i:i+p] = motif_pattern
#save to file as csv
data_scenario1 = data_scenario1.astype(int)
np.savetxt("../data/syntheticdata/scenario1.csv", data_scenario1, delimiter=",")
np.savetxt("../data/syntheticdata/motif_indexes_scenario1.csv", motif_indexes_scenario1, delimiter=",")

In [ ]:
variable_indexes = np.arange(k)
#change the values of the time series in the selected timewindows to the motif pattern
for idx in motif_indexes_scenario1:
    data_scenario1[variable_indexes,idx:idx+p] = motif_pattern

#introduce hint in the data
for idx in motif_indexes_scenario1:
   data_scenario1[variable_indexes,idx-9:idx-9+3] = [5,5,5]

data_scenario1 = data_scenario1.astype(int)
np.savetxt("../data/syntheticdata/scenario1_withhint.csv", data_scenario1, delimiter=",")

#plot multivariate data using subplots in red the planted motifs
fig, axis = plt.subplots(nrows=k, ncols=1, figsize=(10, 10), sharex=True)
for i, variable in enumerate(variable_indexes):
 plt.subplot(len(variable_indexes), 1, i+1)
 #y axis from 1 to 5
 plt.yticks([1,5])
 plt.plot(data_scenario1[variable], linewidth=0.5)
 #plot the planted motifs
 for idx in motif_indexes_scenario1:
    plt.axvline(x=idx, color='r', linestyle='--', linewidth = 2)

plt.xlabel('Time', size=20)
plt.tight_layout()
plt.savefig("../images/data_scenario1_withhint.pdf")
plt.show()

In [ ]:
from keras.preprocessing.sequence import pad_sequences

def create_dataset(data,variable_indexes, lookback_period, step, forecast_period, motif_indexes):
    X1, X2, y = list(), list(),  list() # X1: data, X2: indexes of the motifs, y: distance to the next motif
    for idx in range(len(data[0]) - lookback_period - 1):
        if idx % step != 0:
            continue

        window_end_idx = idx + lookback_period
        forecast_period__end = window_end_idx + forecast_period

        #if doesnt exist more matches after the window
        if not any([window_end_idx < motif_idx for motif_idx in motif_indexes]):
            break

        #motifs_indexes_in_window relative to the window
        motif_indexes_in_window = [motif_idx - idx for motif_idx in motif_indexes if idx <= motif_idx <= window_end_idx]
        motif_indexes_in_forecast_period = [motif_idx for motif_idx in motif_indexes if window_end_idx < motif_idx <= forecast_period__end]

        if motif_indexes_in_forecast_period:
            next_match_in_forecast_period = motif_indexes_in_forecast_period[0]
        else:
            next_match_in_forecast_period = -1  #no match in forward window but exist in the future

        data_window = data[variable_indexes,idx:window_end_idx]

        data_y = -1 
        if next_match_in_forecast_period != -1:
            # index of the next match relative to the last match in the window
            #data_y = next_match_in_forecast_period - motif_indexes_in_window[-1] 
            # index of the next match relative to the end of the window
            data_y = next_match_in_forecast_period - window_end_idx
        
        X1.append(data_window)
        X2.append(motif_indexes_in_window) 
        y.append(data_y) 

    X2 = pad_sequences(X2, padding='post', value=-1, dtype=int)
    return np.array(X1), np.array(X2) , np.array(y)

In [ ]:
#load data
data_scenario1 = np.genfromtxt("../data/syntheticdata/scenario1_withhint.csv", delimiter=",")
data_scenario1 = data_scenario1.astype(int)
data_scenario1 = data_scenario1.reshape((k, n))

motif_indexes_scenario1 = np.genfromtxt("../data/syntheticdata/motif_indexes_scenario1.csv", delimiter=",")
motif_indexes_scenario1= motif_indexes_scenario1.astype(int)


lookback_period = 100 #window size
step = 1 #step size for the sliding window
forecast_period = 50 #forward window size
#x1: past window + masking, x2: indexes of the motif in the window,  y: next relative index of the motif
X1, X2 , y  = create_dataset(data_scenario1,variable_indexes=np.arange(k), lookback_period=lookback_period, step=step, forecast_period=forecast_period, motif_indexes=motif_indexes_scenario1)
print(X1.shape, X2.shape, y.shape)
print(X1[0,0], X2[0], y[0])

X1 = np.transpose(X1, (0, 2, 1)) #reshape input to be [samples, time steps, features]
X2 = X2.reshape(X2.shape[0], X2.shape[1], 1) #reshape input to be [samples, time steps, features]
pattern_array = np.tile(motif_pattern, (X1.shape[0], 1)).reshape(X1.shape[0], p, 1)
X1.shape, X2.shape, pattern_array.shape

In [ ]:
class BlockingTimeSeriesSplit():
  def __init__(self, n_splits):
      self.n_splits = n_splits

  def get_n_splits(self, X, y, groups):
      return self.n_splits

  def split(self, X, y=None, groups=None):
      n_samples = len(X)
      k_fold_size = n_samples // self.n_splits
      indices = np.arange(n_samples)

      margin = 0
      for i in range(self.n_splits):
          start = i * k_fold_size
          stop = start + k_fold_size
          mid = int(0.8 * (stop - start)) + start
          yield indices[start: mid], indices[mid + margin: stop]

In [ ]:
import collections
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback
import pickle

class CVTuner(kt.engine.tuner.Tuner):

    def __init__(self, *args, **kwargs):
        super(CVTuner, self).__init__(*args, **kwargs)
        # cols are metrics-val_loss, val_mae, val_rmse, val_r2
        self.metrics_history = []
        #if exists load metrics history
        if os.path.exists(os.path.join(self.project_dir, "metrics_history.npy")):
            self.metrics_history = np.load(os.path.join(self.project_dir, "metrics_history.npy"), allow_pickle=True)
        

    def run_trial(self, trial, x, y, *args, **kwargs):
        callbacks = kwargs.pop("callbacks", [])
        callbacks.append(EarlyStopping(patience=10, monitor="val_loss", start_from_epoch = 100))
        verbose = kwargs.pop("verbose", 0)
        batch_size = trial.hyperparameters.Choice('batch_size', values=[8,16,32])

        metrics = collections.defaultdict(list)
        cv = BlockingTimeSeriesSplit(n_splits=5)
        trial_metrics = np.zeros((5, 4), dtype=np.float32)

        for i, (train_indices, test_indices) in enumerate(cv.split(y)):
            #split train indices into train and validation
            #if x is list
            if isinstance(x, list):
                X_train = [x_in[train_indices] for x_in in x]
                X_test = [x_in[test_indices] for x_in in x]
            else:
                X_train, X_test = x[train_indices], x[test_indices]
            y_train, y_test = y[train_indices], y[test_indices]

            model = self.hypermodel.build(trial.hyperparameters)
            model.fit(X_train, y_train, validation_split=0.2, epochs=400, batch_size=batch_size, verbose=verbose, callbacks=callbacks)
            test_loss, test_mae, test_rmse, test_r2 = model.evaluate(X_test, y_test) 
            metrics["test_loss"].append(test_loss)
            metrics["test_mae"].append(test_mae)
            metrics["test_rmse"].append(test_rmse)
            metrics["test_r2"].append(test_r2)

            trial_metrics[i] = [test_loss, test_mae, test_rmse, test_r2]
                
        self.save_trial_metrics(trial.trial_id, trial_metrics)
        self.metrics_history.append(trial_metrics)
        print({name: np.mean(values) for name, values in metrics.items()})
        #save cross validation metrics
        return {name: np.mean(values) for name, values in metrics.items()}

    def save_trial_metrics(self, trial_id, metrics):
        fname = os.path.join(self.get_trial_dir(trial_id), "metrics.pickle")
        with open(fname, "wb") as f:
            pickle.dump(metrics, f)

    def save_metrics(self):
        # aggregate metric_history into a matrix (num trials, num folds, num metrics)
        fname = os.path.join(self.project_dir, "metrics_history.npy")
        np.save(fname, np.array(self.metrics_history))
    
    def save_model(self, trial_id, model, step=0):
        fname = os.path.join(self.get_trial_dir(trial_id), "model.pickle")
        with open(fname, "wb") as f:
            pickle.dump(model, f)

    def load_model(self, trial):
        fname = os.path.join(self.get_trial_dir(trial.trial_id), "model.pickle")
        with open(fname, "rb") as f:
            return pickle.load(f)


In [ ]:
from keras.optimizers import Adam
from keras.layers import Dense, Masking, Concatenate, Flatten, Conv1D
from keras import Input
from tqdm.keras import TqdmCallback

class MyHyperModel_CNN(kt.HyperModel):
    def __init__(self, time_series_shape):
        self.time_series_shape = time_series_shape
        #self.masked_series_shape = masked_series_shape
       # self.motif_indexes_shape = motif_indexes_shape
        #self.pattern_shape = pattern_shape

    def build(self, hp):
        series_input = Input(shape=self.time_series_shape) # (observations, time steps, features)
        #masked_series_input = Input(shape=self.masked_series_shape) # (observations, time steps, 1)
        #motif_indexes_input = Input(shape=self.motif_indexes_shape) # (observations, numer_indexes, 1)
        #pattern_input = Input(shape=self.pattern_shape) # (observations, pattern_size, 1)
         
        #motif_indexes_input = Masking(mask_value=-1)(motif_indexes_input)


        conv_1d = Conv1D(filters=hp.Choice('filters', values=[32,64,128]), kernel_size=hp.Choice('kernel_size', values=[3,5]), activation='relu')(series_input)
        series_flatten = Flatten()(conv_1d)
        #masked_series_flatten = Flatten()(conv_1d)
        #motif_indexes_flatten = Flatten()(conv_1d)
        #pattern_input_flatten = Flatten()(pattern_input)


        #concat = Concatenate(axis=1)([series_flatten, motif_indexes_flatten, pattern_input_flatten])

        units_dense1 = hp.Choice('units', values=[64])
        dense1 = Dense(units = units_dense1, activation='relu')(series_flatten)
        dense2 = Dense(units = units_dense1/2, activation='relu')(dense1)
        output = Dense(1)(dense2)
        model = keras.Model(inputs=[series_input], outputs=output)
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
        model.compile(loss='mse', optimizer=Adam(learning_rate=hp_learning_rate), metrics=['mae', 'root_mean_squared_error', 'r2_score'])
        return model

In [ ]:
X1[0,:,:].flatten(), X2[0,:,:].flatten(), pattern_array[0,:,:].flatten(), y[0]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#Masking the data for input, only 1 in motif indexes rest is zero
masking_X1 = np.zeros((X1.shape[0], X1.shape[1], 1))
for i, obs_motif_indexes in enumerate(X2):
    for j, idx in enumerate(obs_motif_indexes):
        masking_X1[i, idx[0]:idx[0]+5, 0] = 1

X1_normalized = X1.copy()
scalers = []
for i in range(X1.shape[2]):
    scaler = MinMaxScaler()
    X1_normalized[:,:,i] = scaler.fit_transform(X1[:,:,i])
    scalers.append(scaler)

X1_normalized_plus_masking = np.concatenate([X1_normalized, masking_X1], axis=2)

tuner = CVTuner(
  hypermodel=MyHyperModel_CNN(time_series_shape= (X1_normalized_plus_masking.shape[1], X1_normalized_plus_masking.shape[2])),
  oracle=kt.oracles.GridSearchOracle(
    objective=kt.Objective("test_loss", "min"),
    max_trials=None),
    directory='../results/syntheticdata',
    project_name='scenario1_cnn_k=3_series+masked_series_normalized')




tuner.search(
        x= X1_normalized_plus_masking,
        y=y,
        verbose=0,
        callbacks=[TqdmCallback(verbose=0)],
    )
tuner.save_metrics()

In [ ]:
tuner.results_summary()

In [ ]:
#get config of the best model
best_trial = tuner.oracle.get_best_trials(1)[0]

print(best_trial.hyperparameters.values)
print(best_trial.metrics.get_best_value("test_mae"))

#load best trial metrics file
cv_metrics = np.load(os.path.join(tuner.get_trial_dir(best_trial.trial_id), "metrics.pickle"), allow_pickle=True)
print(cv_metrics)

df_cv_metrics = pd.DataFrame({
    'Mean': np.mean(cv_metrics, axis=0),
    'STD': np.std(cv_metrics, axis=0)
}, index=['Loss', 'MAE', 'RMSE', 'R2'])
print(df_cv_metrics)

metrics = []
#train test split
mid = int(0.8 * len(np.arange(len(X1))))
X1_train, X1_test = X1[:mid], X1[mid:]
X2_train, X2_test = X2[:mid], X2[mid:]
pattern_array_train, pattern_array_test = pattern_array[:mid], pattern_array[mid:]
y_train, y_test = y[:mid], y[mid:]
model = (MyHyperModel_CNN(time_series_shape=(X1.shape[1], X1.shape[2]), motif_indexes_shape=(X2.shape[1], X2.shape[2]), pattern_shape=(pattern_array.shape[1], pattern_array.shape[2]))).build((best_trial.hyperparameters))
history = model.fit([X1_train, X2_train, pattern_array_train], y_train, validation_split = 0.2,
            epochs=400, batch_size=best_trial.hyperparameters.values["batch_size"],
            verbose=0, callbacks=[TqdmCallback(verbose=0)])
metrics.append(model.evaluate([X1_test, X2_test, pattern_array_test], y_test))


keras.utils.plot_model(model, to_file="../images/best_model_cnn_casestudy_scenario1.png", show_shapes=True)
metrics = np.array(metrics)
print(metrics)

In [ ]:
logs = pd.DataFrame(history.history)

plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.plot(logs["loss"], lw=2, label='training loss')
plt.plot(logs["val_loss"], lw=2, label='validation loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(logs["mae"], lw=2, label='training MAE')
plt.plot(logs["val_mae"], lw=2, label='validation MAE')
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.legend(loc='upper right')
plt.show()

In [ ]:
#plot prediction vs true values
y_pred = model.predict([X1_test, X2_test, pattern_array_test])

residuals = y_test - y_pred.flatten()
plt.scatter(y_pred, residuals)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='-')
plt.title('Residual Plot')
plt.show()